In [3]:
import os
import time
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import gc
sys.path.append('/home/ld-sgdev/yunrui_li/ner_project/brand_recognition_bio_FE/preprocessing')
sys.path.append('/home/ld-sgdev/yunrui_li/ner_project/brand_recognition_bio_FE/py_model')
sys.path.append('../models/')
from clean_helpers import clean_name_for_word_embedding
from utils import init_logging
from data_utils import get_glove_vocab
from data_utils import write_vocab
from data_utils import load_vocab_and_return_word_to_id_dict
from data_utils import export_glove_vectors
import logging
import gc


In [4]:
def pad_1d(array, max_len):
    array = array[:max_len]
    length = len(array)
    padded = array + [0]*(max_len - len(array))
    return padded, length

def encode_word_to_idx(word, word_to_id, vocabulary_set, lowercase = True, allow_unknown = True):
    '''encode a word (string) into id'''

    # 1. preprocess word
    if lowercase:
        word = word.lower()
    if word.isdigit():
        word = NUM

    # 2. get id of word
    if word in vocabulary_set:
        return word_to_id[word]
    else:
        if allow_unknown:
            return word_to_id[UNK]
        else:
            raise Exception("Unknow key is not allowed. Check that your vocab (tags?) is correct")

#--------------------
# setting
#--------------------
TRACE_CODE = False # for tracing funtionality and developing quickly
TRUNCATED = False
LOWERCASE = True
ALLOW_UNKNOWN = True
dim_word = 300
UNK = "$UNK$" # for the word in our own courpus which is unknown in embedding
NUM = "$NUM$" # for the word which is number

base_dir = "../models/data/wordvec"
filename_words_voc = "../models/data/wordvec/words_vocab.txt"
pre_trained_word_embedding_path = "/data/ID_largewv_300_2.txt"
filename_words_vec = "../models/data/wordvec/word2vec.npz".format(dim_word)
log_dir = 'log/' # log path
init_logging(log_dir)

#-------------------
# loading data
#-------------------
if TRACE_CODE == True:
    df = pd.read_csv('../data/processed/mobile_training.csv', nrows = 19)
else:
    df = pd.read_csv('../data/processed/mobile_training.csv')

s = time.time()

num_sentences = df['item_name'].nunique()
seq_len_distribution = df.groupby('item_name').tokens.apply( lambda x : len(x.tolist())).to_frame('seq_len').reset_index()

if TRUNCATED == False:
    max_seq_length = seq_len_distribution.seq_len.max()
else:
    max_seq_length = 100

# preprocessing
df['clean_tokens'] = df.tokens.apply(lambda x: clean_name_for_word_embedding(x) if type(x)==str else x)
if LOWERCASE:
	df['clean_tokens'] = df.clean_tokens.apply(lambda x: x.lower() if type(x)==str else x)

e = time.time()
logging.info('it spend {} mins on preprocessing'.format( (e-s) / 60.0))

it spend 0.06906096537907919 mins on preprocessing
it spend 0.06906096537907919 mins on preprocessing


In [5]:
df['clean_tokens'] = df.clean_tokens.astype(str)

In [6]:
df.clean_tokens.value_counts()

                     20124
samsung               5786
galaxy                3731
black                 3685
for                   3486
ram                   2931
xiaomi                2697
4g                    2447
battery               2322
oem                   2108
smartphone            2095
garansi               2089
gold                  2041
phone                 2020
resmi                 1905
case                  1900
lg                    1874
gb                    1872
redmi                 1857
nokia                 1666
motorola              1656
blackberry            1641
dual                  1578
cover                 1547
note                  1517
32gb                  1514
rom                   1492
16gb                  1441
original              1426
htc                   1354
                     ...  
18gb4g                   1
blacktftandroid40        1
a2dp                     1
sisi                     1
sphm850                  1
432gbcamera              1
c

In [13]:
len(df)

253553

In [3]:
df = pd.read_csv('../data/processed/mobile_training.csv')


In [4]:
df.

,item_name,tokens,is_brand,is_valid
0,Samsung Galaxy J1 Ace 2016 SM-J111F 8GB - White,Samsung,2,train
1,Samsung Galaxy J1 Ace 2016 SM-J111F 8GB - White,Galaxy,0,train
2,Samsung Galaxy J1 Ace 2016 SM-J111F 8GB - White,J1,0,train
3,Samsung Galaxy J1 Ace 2016 SM-J111F 8GB - White,Ace,0,train
4,Samsung Galaxy J1 Ace 2016 SM-J111F 8GB - White,2016,0,train
5,Samsung Galaxy J1 Ace 2016 SM-J111F 8GB - White,SM-J111F,0,train
6,Samsung Galaxy J1 Ace 2016 SM-J111F 8GB - White,8GB,0,train
7,Samsung Galaxy J1 Ace 2016 SM-J111F 8GB - White,-,0,train
8,Samsung Galaxy J1 Ace 2016 SM-J111F 8GB - White,White,0,train
9,Blackview BV8000 Pro RAM 6GB 64GB IP68 Wa...,Blackview,2,train


In [9]:
UNK = "$UNK$"
NUM = "$NUM$"
NONE = "O"

In [15]:
'$UNK$' in vocabulary_set

False

In [17]:
len(vocabulary_set)

6629250

In [22]:
d = dict({'A':1,'B':2}) 
d

{'A': 1, 'B': 2}

In [24]:
d[UNK]

KeyError: '$UNK$'